In [3]:
from Functions_1 import normalize_data, time_series_CV_split, wrapper_feature_selector, train_and_predict, warn
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.preprocessing import MinMaxScaler 
from time import time
from math import sqrt
from sklearn.linear_model import BayesianRidge
from matplotlib import pyplot
import plotly.offline as py
import plotly.graph_objs as go
import warnings
warnings.warn = warn
py.init_notebook_mode(connected=True)
%matplotlib inline

# Train Set
train_data = pd.read_csv('/Users/Andy/Desktop/Bitcoin/Data/Train_data_after_EDA.csv')
print('n_features:', len(train_data.iloc[0]))
print('n_samples:', len(train_data))
train_data.head()

n_features: 11
n_samples: 365


,date,close,volume,BCHAIN-DIFF,BCHAIN-AVBLS,BCHAIN-MIREV,BCHAIN-CPTRA,BCHAIN-NTRAN,BCHAIN-HRATE,BCHAIN-CPT,BCHAIN-NTRBL
0,1/1/18,13657.2,10291200000,1.870000e+12,1.037057,35435185.86,125.970638,290422,15177350.25,122.012747,1781.730061
1,2/1/18,14982.1,16846600192,1.920000e+12,1.043383,32334328.99,138.227164,241757,14975580.96,133.747230,1549.724359
2,3/1/18,15201.0,16871900160,1.930000e+12,1.041368,40553327.64,122.635624,340980,16415540.67,118.931690,1994.035088
3,4/1/18,15599.2,21783199744,1.930000e+12,1.065513,39612658.08,103.108719,395963,15071578.27,100.041312,2522.057325
4,5/1/18,17429.5,23840899072,1.930000e+12,1.065833,42527795.29,102.933856,425008,16127548.73,100.063517,2529.809524


In [4]:
# Test Set
test_data = pd.read_csv('/Users/Andy/Desktop/Bitcoin/Data/Test_data_after_EDA.csv')
print('n_features:', len(test_data.iloc[0]))
print('n_samples:', len(test_data))
test_data.head()

n_features: 11
n_samples: 181


,date,close,volume,BCHAIN-DIFF,BCHAIN-AVBLS,BCHAIN-MIREV,BCHAIN-CPTRA,BCHAIN-NTRAN,BCHAIN-HRATE,BCHAIN-CPT,BCHAIN-NTRBL
0,1/1/19,3843.52,4324200990,5.250000e+12,0.888394,7406437.589,28.520962,259684,43291796.76,28.520962,1675.380645
1,2/1/19,3943.41,5244856836,5.620000e+12,0.801779,7030739.129,29.953090,234725,41615985.27,29.953090,1575.335570
2,3/1/19,3836.74,4530215219,5.620000e+12,0.947861,7368988.356,27.122182,271696,42174589.10,27.122182,1799.311258
3,4/1/19,3857.72,4847965467,5.620000e+12,0.966222,7564081.637,25.991979,291016,43291796.76,25.991979,1877.522581
4,5/1/19,3845.19,5137609824,5.620000e+12,0.959199,7194486.330,25.533007,281772,41615985.27,25.533007,1891.087248


In [5]:
combined_data = pd.concat([train_data, test_data], ignore_index=True) # use data from 01/2018 onwards only - best stationarity
combined_data['Price'] = combined_data['close'].shift(-1) # Dependent variable Y

In [6]:
# Combined train and test sets
combined_data = pd.concat([train_data, test_data], ignore_index=True) # use data from 01/2018 onwards only - best stationarity
combined_data['Price'] = combined_data['close'].shift(-1) # Dependent variable Y
print('n_features:', len(combined_data.iloc[0]))
print('n_samples:', len(combined_data))
combined_data = combined_data.iloc[:-1,1:]
combined_data.tail()

n_features: 12
n_samples: 546


,close,volume,BCHAIN-DIFF,BCHAIN-AVBLS,BCHAIN-MIREV,BCHAIN-CPTRA,BCHAIN-NTRAN,BCHAIN-HRATE,BCHAIN-CPT,BCHAIN-NTRBL,Price
540,11790.92,24879684533,7.410000e+12,1.215810,25117669.41,64.679250,388342,65193212.50,64.679250,2194.022599,13016.23
541,13016.23,45105733173,7.410000e+12,1.245764,22019741.67,60.931532,361385,53775192.23,60.931532,2475.239726,11182.81
542,11182.81,39977475223,7.410000e+12,1.240849,28500796.50,71.319031,399624,60773333.69,71.319031,2421.963636,12407.33
543,12407.33,35087757766,7.870000e+12,1.233318,23373593.45,64.737830,361050,56010016.17,64.737830,2542.605634,11959.37
544,11959.37,29923961128,7.930000e+12,1.255488,25894830.35,63.608971,407094,64293187.57,63.608971,2497.509202,10817.16


In [8]:
# Feature Meta Subset
subset = [8, 7, 6, 5, 4, 3, 2, 1, 0] 

# Split train data into X (features) and Y (dependent variable)
data = combined_data.values 
Y_train = data[:-181,-1].reshape(-1,1) 
X_train = data[:-181,:-1]

# Training Validation samples size (1/10/18 - 31/12/18)
n_validation = 92

In [9]:
from sklearn.pipeline import Pipeline
pipe = pipe = Pipeline([
    ('normalize', MinMaxScaler()),
    ('extrees', DecisionTreeRegressor())
])
pipe

Pipeline(memory=None,
     steps=[('normalize', MinMaxScaler(copy=True, feature_range=(0, 1))), ('extrees', DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'))])

In [12]:
%%time

sc = SparkContext.getOrCreate()

param_grid = [{"extrees__max_features": ['auto',5,10,15],
              "extrees__random_state": [100],
              "extrees__max_depth": [10,None],
              "extrees__min_samples_split": [0.8,0.9,1.0,2],
              "extrees__min_samples_leaf": [0.3,0.4,0.5,1]
        }]
split = time_series_CV_split(len(X_train),n_validation,0)
gs = GridSearchCV(sc, pipe, param_grid, cv=split, scoring=make_scorer(mean_squared_error,greater_is_better=False), verbose=1) 
gs.fit(X_train,Y_train.reshape(-1,))
print("Best RMSE:", str(gs.cv_results_['mean_test_score'][gs.best_index_]))
print('Best Parameters: ',gs.cv_results_['params'][gs.best_index_])
sc.stop()

Fitting 92 folds for each of 128 candidates, totalling 11776 fits


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4416 in stage 0.0 failed 1 times, most recent failure: Lost task 4416.0 in stage 0.0 (TID 4416, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/anaconda3/lib/python3.5/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/anaconda3/lib/python3.5/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/anaconda3/lib/python3.5/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 390, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/anaconda3/lib/python3.5/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return